# I. Introduction to Cohort Analysis

## 1.1. Overall Understanding to Cohort Analysis

- Definition: 
    + Cohort analysis is a method of customer segmentation that help businesses understand their customers. Cohort analysis focuses on how a customer's behavior changes over time, while RFM analysis focuses on a customer's current behavior. 
- Focus: 
    + How a customer's behavior changes over time 
- Goal: 
    + Identify trends and patterns in customer behavior 
    + Compare metrics across product lifecycle
    + Compare metrics across customer lifecycle
- How it works: 
    + Groups customers into mutually exclusive **cohorts** based on when they were acquired and tracks their behavior over time 
- When it's useful: 
    + For understanding how a group of customers evolves over time
- Benefits of cohort analysis:
    + Helps identify trends and patterns in customer behavior
    + Helps identify key metrics like retention rate and upsell rate
    + Can be used to improve customer experience and retention

## 1.2. Types of Cohorts

- **Time Cohorts:**
    + Time Cohorts are customers who signed up for a product or service during a particular time frame. 
    + Analyzing these cohorts shows the customers’ behavior depending on the time they started using the company’s products | services. 
    + The time may be monthly or quarterly, even daily.

- **Behavior Cohorts:**
    + Behavior Cohorts are customers who purchased a product or subscribed to a service in the past. 
    + It groups customers by the type of product or service they signed up. 
    + Customers who signed up for basic level services might have different needs than those who signed up for advanced services. 
    + Understanding the needs of the various cohorts can help a company design custom-made services or products for particular segments.
    
- **Size Cohorts:**
    + Size Cohorts refer to the various sizes of customers who purchase company’s products or services. 
    + This categorization can be based on the amount of spending in some period of time after acquisition, or the product type that the customer spent most of their order amount in some period of time.

## 1.3. Time Cohort Example

In [57]:
# import libs & datasets
import pandas as pd
import datetime as dt
online = pd.read_csv('data/E-Commerce Data.csv', encoding='ISO-8859-1')

# include only UK
online = online[online['Country'] == 'United Kingdom']

# include only date, price, customerid 
online = online[['InvoiceDate','UnitPrice','CustomerID']]

# remove duplicates
online.drop_duplicates(inplace=True)

# Drop rows with missing CustomerID
online = online[~online['CustomerID'].isnull()]
online.reset_index(drop=True, inplace=True)

# datetime conversion
online['InvoiceDate'] = pd.to_datetime(online['InvoiceDate'],format='%m/%d/%Y %H:%M')

# view first 5 rows
online.head()


,InvoiceDate,UnitPrice,CustomerID
0,2010-12-01 08:26:00,2.55,17850.0
1,2010-12-01 08:26:00,3.39,17850.0
2,2010-12-01 08:26:00,2.75,17850.0
3,2010-12-01 08:26:00,7.65,17850.0
4,2010-12-01 08:26:00,4.25,17850.0


In [58]:
# preview data information
online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159858 entries, 0 to 159857
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceDate  159858 non-null  datetime64[ns]
 1   UnitPrice    159858 non-null  float64       
 2   CustomerID   159858 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 3.7 MB


In [59]:
# get first day of the month from a given date object
def get_month(datetime: dt.datetime) -> dt.datetime:
    '''get first day of the month from a given date object: datetime64[ns]'''
    return dt.datetime(
        year=datetime.year
        ,month=datetime.month
        ,day=1
    )

# assign acquisition month cohort
online['InvoiceMonth'] = online['InvoiceDate'].apply(get_month)

# preview first 5 rows
online.head()

,InvoiceDate,UnitPrice,CustomerID,InvoiceMonth
0,2010-12-01 08:26:00,2.55,17850.0,2010-12-01
1,2010-12-01 08:26:00,3.39,17850.0,2010-12-01
2,2010-12-01 08:26:00,2.75,17850.0,2010-12-01
3,2010-12-01 08:26:00,7.65,17850.0,2010-12-01
4,2010-12-01 08:26:00,4.25,17850.0,2010-12-01


In [60]:
# assign the smallest InvoiceMonth value of each customer to each record regarding the same customer ID
online['CohortMonth'] = online.groupby('CustomerID')['InvoiceMonth'].transform('min')

# preview last 5 rows
online.tail()

,InvoiceDate,UnitPrice,CustomerID,InvoiceMonth,CohortMonth
159853,2011-12-09 12:31:00,0.95,15804.0,2011-12-01,2011-05-01
159854,2011-12-09 12:49:00,2.95,13113.0,2011-12-01,2010-12-01
159855,2011-12-09 12:49:00,1.25,13113.0,2011-12-01,2010-12-01
159856,2011-12-09 12:49:00,8.95,13113.0,2011-12-01,2010-12-01
159857,2011-12-09 12:49:00,7.08,13113.0,2011-12-01,2010-12-01


In [61]:
# Assign Time Offset Value Process

# def helper function
def get_date_as_int(dataframe: pd.DataFrame, column: str) -> tuple:
    '''
    Function to extract year, month and day as integer values from a datetime object. 
    
    Required pandas as pd and datetime as dt packages

    Return tuple(year: Series[int], month: Series[int], day: Series[int])
    '''
    year = dataframe[column].dt.year
    month = dataframe[column].dt.month
    day = dataframe[column].dt.day
    return year, month, day

# extract year, month from InvoiceMonth, CohortMonth variables
invoice_year, invoice_month, _ = get_date_as_int(online,'InvoiceMonth')
cohort_year, cohort_month, _ = get_date_as_int(online,'CohortMonth')

# Calculate time offset including year and month
years_diff = invoice_year - cohort_year
months_diff = invoice_month - cohort_month

# create cohort index
online['CohortIndex'] = years_diff * 12 + months_diff + 1

# excluding unnecessary columns
# online = online[['CustomerID','CohortMonth','CohortIndex']]

# preview last 5 rows
online.tail()

,InvoiceDate,UnitPrice,CustomerID,InvoiceMonth,CohortMonth,CohortIndex
159853,2011-12-09 12:31:00,0.95,15804.0,2011-12-01,2011-05-01,8
159854,2011-12-09 12:49:00,2.95,13113.0,2011-12-01,2010-12-01,13
159855,2011-12-09 12:49:00,1.25,13113.0,2011-12-01,2010-12-01,13
159856,2011-12-09 12:49:00,8.95,13113.0,2011-12-01,2010-12-01,13
159857,2011-12-09 12:49:00,7.08,13113.0,2011-12-01,2010-12-01,13


In [62]:
# calculate monthly active customers from each cohort
cohort_data = online.groupby(['CohortMonth','CohortIndex'])['CustomerID'].apply(pd.Series.nunique).reset_index()
cohort_data

,CohortMonth,CohortIndex,CustomerID
0,2010-12-01,1,871
1,2010-12-01,2,322
2,2010-12-01,3,291
3,2010-12-01,4,329
4,2010-12-01,5,308
...,...,...,...
86,2011-10-01,2,86
87,2011-10-01,3,40
88,2011-11-01,1,296
89,2011-11-01,2,41


In [63]:
# time cohorts table
cohort_counts = cohort_data.pivot(
    index='CohortMonth'
    ,columns='CohortIndex'
    ,values='CustomerID'
)
cohort_counts

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2010-12-01,871.0,322.0,291.0,329.0,308.0,345.0,327.0,304.0,306.0,346.0,320.0,429.0,238.0
2011-01-01,362.0,84.0,101.0,89.0,124.0,106.0,95.0,94.0,114.0,127.0,131.0,54.0,NaN
2011-02-01,339.0,85.0,65.0,95.0,96.0,86.0,88.0,96.0,94.0,106.0,33.0,NaN,NaN
2011-03-01,408.0,79.0,107.0,88.0,95.0,70.0,107.0,97.0,119.0,38.0,NaN,NaN,NaN
2011-04-01,276.0,62.0,61.0,60.0,57.0,64.0,64.0,73.0,23.0,NaN,NaN,NaN,NaN
2011-05-01,252.0,58.0,43.0,43.0,54.0,60.0,67.0,25.0,NaN,NaN,NaN,NaN,NaN
2011-06-01,207.0,44.0,34.0,51.0,53.0,67.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN
2011-07-01,172.0,35.0,33.0,40.0,48.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08-01,140.0,37.0,32.0,36.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# DataCamp data, same logic but we use this dataset instead for data consistency
cohort_counts = pd.read_csv('data/cohort_counts.csv',index_col=0)
cohort_counts

,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2010-12-01,716,246.0,221.0,251.0,245.0,285.0,249.0,236.0,240.0,265.0,254.0,348.0,172.0
2011-01-01,332,69.0,82.0,81.0,110.0,90.0,82.0,86.0,104.0,102.0,124.0,45.0,NaN
2011-02-01,316,58.0,57.0,83.0,85.0,74.0,80.0,83.0,86.0,95.0,28.0,NaN,NaN
2011-03-01,388,63.0,100.0,76.0,83.0,67.0,98.0,85.0,107.0,38.0,NaN,NaN,NaN
2011-04-01,255,49.0,52.0,49.0,47.0,52.0,56.0,59.0,17.0,NaN,NaN,NaN,NaN
2011-05-01,249,40.0,43.0,36.0,52.0,58.0,61.0,22.0,NaN,NaN,NaN,NaN,NaN
2011-06-01,207,33.0,26.0,41.0,49.0,62.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN
2011-07-01,173,28.0,31.0,38.0,44.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08-01,139,30.0,28.0,35.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Question: How many customers have made their first transaction in January 2011 ?

In [9]:
cohort_counts.loc[
    # transaction date
    '2011-01-01'
    # the n th transaction, in this case it is the first trans so 1
    ,'1'
]

332

## 1.4. Cohort Metrics

### 1.4.1. Retention Rate

### 1.4.2. Average Quantity

## 1.5. Visualizing Cohort Analysis

# II. Case Study 